 <h1> Twitter Sentiment Analysis </h1>
 This notebook is used for creating a simple Logistic regression Model meant for  classifying the tweets or texts either as a negative statement or a postitive statement. The result is either 0 or 1. 
 

Importing the necessary libraries  


In [209]:
import pandas as pd 
import numpy as np 
import re 
import nltk 
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
#from sklearn.model_selection import cross_val_score_val, cross_validate
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import GridSearchCV
import contractions
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Reading the data stored in csv format.

In [210]:
data = pd.read_csv('tweets.csv')
data.tail()

,id,tweet,label
7915,7916,Live out loud #lol #liveoutloud #selfie #smile...,1
7916,7917,We would like to wish you an amazing day! Make...,1
7917,7918,Helping my lovely 90 year old neighbor with he...,1
7918,7919,Finally got my #smart #pocket #wifi stay conne...,1
7919,7920,Apple Barcelona!!! #Apple #Store #BCN #Barcelo...,1


In [211]:
data.head()

,id,tweet,label
0,1,#fingerprint #Pregnancy Test https://goo.gl/h1...,1
1,2,Finally a transparant silicon case ^^ Thanks t...,1
2,3,We love this! Would you go? #talk #makememorie...,1
3,4,I'm wired I know I'm George I was made that wa...,1
4,5,What amazing service! Apple won't even talk to...,0


In [246]:
data.shape

(7920, 3)

In [247]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7920 non-null   int64 
 1   tweet   7920 non-null   object
 2   label   7920 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 185.8+ KB


In [245]:
data.label.value_counts() #understanding label distribution

label
1    5894
0    2026
Name: count, dtype: int64

In [215]:
df = data[['tweet' , 'label']] #

In [248]:
df.head(2)

,tweet,label,tweets_without_stop_words
0,#fingerprint #Pregnancy Test https://goo.gl/h1...,1,fingerprint pregnancy test android apps beauti...
1,Finally a transparant silicon case ^^ Thanks t...,1,finally a transparant silicon case thanks to m...


In [217]:
# Stop words Removal
import nltk
from nltk.corpus import stopwords

In [221]:
combined_stopwords = nltk_stopwords.union(sklearn_stopwords)


In [222]:

domain_stopwords = ['phone', 'mobile', 'twitter', 'rt', 'com', 'follow']
final_stopwords = domain_stopwords + list(combined_stopwords)

In [249]:
########################## function to clean the tweets#######################

# we are keeping the stop words as well#
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def tweet_cleaner_without_stopwords(text):
    new_text = re.sub(r"'s\b", " is", text)
    new_text = re.sub("#", "", new_text)
    new_text = re.sub("@[A-Za-z0-9]+", "", new_text)
    new_text = re.sub(r"http\S+", "", new_text)
    new_text = contractions.fix(new_text)    
    new_text = re.sub(r"[^a-zA-Z]", " ", new_text)    
    new_text = new_text.lower().strip()
    
    
    ###########if you want to remove stopwords and experiment, uncomment these two lines ####
    #     new_text = [token for token in new_text.split() if token not in combined_stopwords]
    
    #     new_text = [token for token in new_text if len(token)>2]
    ###############
    cleaned_text = ''
    for token in new_text.split():
        cleaned_text = cleaned_text + lemmatizer.lemmatize(token) + ' '
    
    return cleaned_text

In [250]:
df['tweets_without_stop_words'] = df['tweet'].apply(tweet_cleaner_without_stopwords)

In [256]:
df[['tweet',"tweets_without_stop_words", 'label']].head()

,tweet,tweets_without_stop_words,label
0,#fingerprint #Pregnancy Test https://goo.gl/h1...,fingerprint pregnancy test android apps beauti...,1
1,Finally a transparant silicon case ^^ Thanks t...,finally a transparant silicon case thanks to m...,1
2,We love this! Would you go? #talk #makememorie...,we love this would you go talk makememories un...,1
3,I'm wired I know I'm George I was made that wa...,i am wired i know i am george i wa made that w...,1
4,What amazing service! Apple won't even talk to...,what amazing service apple will not even talk ...,0


Examplar Label and its respective tweet 

In [257]:
df["label"][1224]

0

In [258]:
df["tweets_without_stop_words"][1224]

'what is next apple do we have to write our apps in machine code will the only supported keyboard be the iphone is one '

In [259]:
df.head(2)

,tweet,label,tweets_without_stop_words
0,#fingerprint #Pregnancy Test https://goo.gl/h1...,1,fingerprint pregnancy test android apps beauti...
1,Finally a transparant silicon case ^^ Thanks t...,1,finally a transparant silicon case thanks to m...


In [260]:
df.tweet[12]#raw tweet 

'Contemplating giving in to the iPhone bandwagon simply because #Cellcom has no new Androids #depressing #IDontWantTo'

In [230]:
df.tweets_without_stop_words[12] #cleaned tweet

'contemplating giving in to the iphone bandwagon simply because cellcom ha no new android depressing idontwantto '

### Model Creation and Cross validation

In [231]:
#Model Building Process and cross validation 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold


X = df['tweets_without_stop_words']
y = df['label']

# we want to include only those words in the vocab which have min df of 5,
# means select only those words which occur ATLEAST in 5 documents!! 
# AND SELECT the TOP 1000 FEATURES ONLY to build the model
TFIDF = TfidfVectorizer(stop_words=final_stopwords,  min_df=5, max_features=1000)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

LR1 = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.4)
CV_pipe = Pipeline([('TFIDF', TFIDF) , ('LR', LR1)] )
results = cross_validate(CV_pipe, X, y, cv=kfold, scoring='accuracy', return_train_score=True)

# print(results['train_score'])
print(np.round((results['train_score'].mean())*100, 2), np.round((results['train_score'].std())*100, 2)) 

# print(results['test_score'])
print(np.round((results['test_score'].mean())*100, 2), np.round((results['test_score'].std())*100, 2)) 

TFIDF.fit_transform(X)
len(TFIDF.vocabulary_)

86.82 0.14
85.24 0.84


1000

### Final Model for saving the model pickle file and use for predictions.

In [232]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import AdaBoostClassifier


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['tweets_without_stop_words'], df['label'], test_size=0.3, random_state=42)

# Define the TF-IDF vectorizer and Logistic Regression model
tfidf_vectorizer = TfidfVectorizer( min_df=5, max_features=1000)
logistic_regression_model = LogisticRegression(class_weight='balanced')

# Create a pipeline that first transforms the data using TF-IDF and then applies Logistic Regression
pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('logistic_regression', logistic_regression_model)
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
predictions = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

In [265]:
print("accuracy :" , accuracy)

accuracy : 0.8817340067340067


In [276]:
print("#############Classification Report#############\n###############################################")
print(report)

#############Classification Report#############
###############################################
              precision    recall  f1-score   support

           0       0.72      0.92      0.81       641
           1       0.97      0.87      0.91      1735

    accuracy                           0.88      2376
   macro avg       0.84      0.89      0.86      2376
weighted avg       0.90      0.88      0.89      2376



### Saving the model in pickle file 

In [ ]:
import pickle

# `pipeline` is your trained pipeline that includes the TF-IDF vectorizer and logistic regression model
with open('models/LRmodel.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

###  Testing the model pickle file

In [288]:
# Read the saved model and use it for prediction 
with open('models/LRmodel.pkl', 'rb') as file:
    model = pickle.load(file)

In [289]:
tweet_to_predict = '#super ? greAT feeling#$  '
ts = tweet_cleaner_without_stopwords(tweet_to_predict)
prediction = model.predict([ts])
print(tweet_to_predict,"-->",ts, "-->",prediction[0])

#super ? greAT feeling#$   --> super great feeling  --> 1


In [290]:
tweet_to_predict = 'what is next apple do we have to write our apps in machine code will the only supported keyboard be the iphone is one '
ts = tweet_cleaner_without_stopwords(tweet_to_predict)
prediction = model.predict([ts])
print(tweet_to_predict,"-->",ts, "-->",prediction[0])

what is next apple do we have to write our apps in machine code will the only supported keyboard be the iphone is one  --> what is next apple do we have to write our apps in machine code will the only supported keyboard be the iphone is one  --> 0


The results of the models may not be very precise and correct. Since this is a basic model. 
Is is important to perform data analysis and try tuning your model in a way to get the best performance out of it before deploying it over the server. You can experiment with the same and use the model which is bests for the data and the problem statement.

In [285]:

#Try Using different methods and models and try comparing the results and models.

# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import MultinomialNB

# # Create instances of different classification algorithms
# svm_model = SVC(class_weight='balanced')
# rf_model = RandomForestClassifier(class_weight='balanced')
# nb_model = MultinomialNB()

# # Create pipelines for each algorithm
# svm_pipeline = Pipeline([
#     ('tfidf', tfidf_vectorizer),
#     ('svm', svm_model)
# ])

# rf_pipeline = Pipeline([
#     ('tfidf', tfidf_vectorizer),
#     ('random_forest', rf_model)
# ])

# nb_pipeline = Pipeline([
#     ('tfidf', tfidf_vectorizer),
#     ('naive_bayes', nb_model)
# ])

# # Train and evaluate each model
# for pipeline, model_name in zip([svm_pipeline, rf_pipeline, nb_pipeline], ['SVM', 'Random Forest', 'Naive Bayes']):
#     # Train the model
#     pipeline.fit(X_train, y_train)

#     # Make predictions
#     predictions = pipeline.predict(X_test)

#     # Evaluate the model
#     accuracy = accuracy_score(y_test, predictions)
#     report = classification_report(y_test, predictions)
    
#     # Print the classification report
#     print(f"Classification Report for {model_name}:")
#     print(report)

In [241]:
prediction[0]

0